In [4]:
import pandas as pd
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

In [7]:
def retrieve_news(tickers):
    
    url = "https://finviz.com/quote.ashx?t="

    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36"
    }

    news_tables = {}

    # Loop through each ticker and fetch the corresponding webpage html code
    for ticker in tickers:
        url_ticker = url + ticker
        req = Request(url_ticker, headers=headers)
        response = urlopen(req)
        
        soup = BeautifulSoup(response.read(), "html.parser")
        
        # Find the table with id 'news-table
        news_table = soup.find(id="news-table")
        news_tables[ticker] = news_table


    parsed_data = []
    #
    for ticker, news_table in news_tables.items():
        # List of all the table rows elements
        news_rows = news_table.findAll("tr")
        for row in news_table.findAll("tr"):
            if row.a is None or row.td is None:
                continue

            title = row.a.get_text()
            date_time = row.td.text.strip().split()
            #Sometimes timestamp has date soemtimes it has only time
            if len(date_time) == 1:
                date = None
                time = date_time[0]
            else:
                date = date_time[0]
                time = date_time[1]
                        
            parsed_data.append([ticker, date, time, title])

    # Convert the parsed data list into pd a DataFrame
    df = pd.DataFrame(parsed_data, columns=["ticker", "date", "time", "title"])
    return df

In [10]:
tickers = ["AAPL", "NVDA", "GOOGL"]
finviz_data = retrieve_news(tickers)
finviz_data.head()

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



,ticker,date,time,title
0,AAPL,Today,06:26AM,Greg Abel's First Significant Move Since Warre...
1,AAPL,None,06:11AM,"Taiwan Semiconductor Shrinks Old Chip Lines, G..."
2,AAPL,None,06:09AM,Lenovo looking to partner with multiple AI mod...
3,AAPL,None,04:40AM,Apple and Dell supplier Pegatron expects US pl...
4,AAPL,None,04:32AM,Apple supplier Pegatron expects US plant to be...


In [2]:
#Use gemini AI to make a recommendation / sentiment.
from google import genai
from dotenv import load_dotenv
import os

In [13]:
def summarize(ticker, titles_list):
    load_dotenv("API_KEY.env") 
    api_key = os.getenv("GEMINI_API_KEY")

    if not api_key:
        raise ValueError("GEMINI_API_KEY not found in environment variables")

    client = genai.Client(api_key=api_key)

    # Format titles nicely for the AI
    titles_text = "\n".join([f"- {t}" for t in titles_list])

    prompt = f"""
You are a financial news analyst.

Summarise the overall news sentiment and key themes for ticker: {ticker}

News headlines:
{titles_text}

Return:
1) 3-5 bullet summary of key themes
2) sentiment (Bullish / Neutral / Bearish)
3) biggest risk mentioned
4) biggest opportunity mentioned
5) 1-sentence takeaway for an investor
"""

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )

    return response.text

In [14]:
for ticker, group in finviz_data.groupby("ticker"):
    titles_list = group["title"].dropna().tolist()

    summary = summarize(ticker, titles_list)
    print(f"\n===== {ticker} =====")
    print(summary)


===== AAPL =====
Here's a summary of the news sentiment and key themes for AAPL based on the provided headlines:

1.  **Key Themes:**

*   **AI Integration & Wearables:** Apple is actively exploring AI, with reports of a revamped Siri chatbot powered by Gemini and a potential AI wearable pin, positioning them in a wearable arms race with Chinese competitors.
*   **China Market Performance:** Apple is showing strong growth in the Chinese smartphone market, retaining leadership and experiencing significant shipment increases.
*   **CEO Succession:**  Increased focus on John Ternus and his expanding role suggests he's a potential candidate for the CEO position.
*   **Supplier News:** Updates on Apple's suppliers, including Pegatron's US plant completion and potential issues for South Korean suppliers.
*   **Earnings & Analyst Outlook:**  Analysts have varied outlooks leading up to earnings, with some seeing a buying opportunity and others discussing potential underperformance.

2.  **Sen